# Experiment 6: Explicit Priors with Pixel2Mesh++

In [2]:
import pandas as pd

In [8]:
results = pd.read_csv("../eval/TestEvaluation/mesh_eval_results.csv")

In [9]:
relevant_models = [
    "refine_p2mp_augmentation_2_inputs_use_spherical_prior",
    "refine_p2mpp_augmentation_2_inputs_use_mean_prior",
    "refine_p2mpp_augmentation_2_inputs",
    # extend with Pixel2Mesh++ and PixelNeRF names as in the CSV file
]

filtered = results[results["model"].isin(relevant_models)]

aggregated = filtered.groupby("model", as_index=False).agg(
    mean_chamfer         = ("chamfer",     "mean"),
    std_chamfer          = ("chamfer",     "std"),
    mean_fscore_1        = ("fscore_1",    "mean"),
    std_fscore_1         = ("fscore_1",    "std"),
    mean_fscore_2_5      = ("fscore_2_5",  "mean"),
    std_fscore_2_5       = ("fscore_2_5",  "std"),
    mean_fscore_5        = ("fscore_5",    "mean"),
    std_fscore_5         = ("fscore_5",    "std"),
    mean_iou             = ("voxel_iou",   "mean"),
    std_iou              = ("voxel_iou",   "std"),
)

aggregated

,model,mean_chamfer,std_chamfer,mean_fscore_1,std_fscore_1,mean_fscore_2_5,std_fscore_2_5,mean_fscore_5,std_fscore_5,mean_iou,std_iou
0,refine_p2mp_augmentation_2_inputs_use_spherica...,0.052072,0.018641,0.202197,0.091306,0.676670,0.126154,0.880312,0.105855,0.430559,0.236246
1,refine_p2mpp_augmentation_2_inputs,0.051909,0.022576,0.212900,0.090377,0.690881,0.155863,0.884539,0.124626,0.406086,0.234189
2,refine_p2mpp_augmentation_2_inputs_use_mean_prior,0.053858,0.019057,0.192833,0.056162,0.666118,0.115554,0.875149,0.104895,0.758664,0.103869
